In [1]:
from collections import Counter
from flask import Flask, request, render_template, jsonify
import pickle
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import argparse
import time
from splinter import Browser
import time
from collections import defaultdict
import json
import re
import requests
import boto3
from boto3.dynamodb.conditions import Key, Attr
import json
from decimal import Decimal

In [2]:
%load_ext autoreload
%autoreload 2

In [68]:
from src.cleaner import Cleaner 
import src.fantasy_scoring as fs
from src.db_helper import DB_helper

In [71]:
#clnr = Cleaner()
db = DB_helper()

reading env variables


In [13]:
lag = "_06"
lag_cols = [col+lag for col in clnr.lag_cols]

In [14]:
lag_cols

['minutes_06',
 'dk_score_06',
 'dk_per_min_06',
 'dk_no_fg_06',
 'dk_no_fg_per_min_06',
 'assists_06',
 'assists_per_min_06',
 'blocks_06',
 'blocks_per_min_06',
 'or_per_min_06',
 'dr_per_min_06',
 'steals_per_min_06',
 'turnovers_per_min_06',
 'doubles_06',
 'fg_pct_06',
 'ft_pct_06',
 'two_pct_06',
 'three_pct_06',
 'game_score_06',
 'attempted_field_goals_06',
 'attempted_three_point_field_goals_06',
 'w_06']

In [72]:
slug = 'curryst01'
resp = db.get_player(slug,lag_cols)
print(len(resp))


595


In [75]:
resp.head()

,assists_06,assists_per_min_06,attempted_field_goals_06,attempted_three_point_field_goals_06,blocks_06,blocks_per_min_06,date,dk_no_fg_06,dk_no_fg_per_min_06,dk_per_min_06,dk_score_06,doubles_06,dr_per_min_06,fg_pct_06,ft_pct_06,game_score_06,minutes_06,or_per_min_06,slug,steals_per_min_06,three_pct_06,turnovers_per_min_06,two_pct_06,w_06
0,8,0.185,16,8,0,0,2013-01-02,21.5,0.498,1.286,55.5,1,0.116,0.688,0.75,28.4,43.15,0.023,curryst01,0.046,0.75,0.093,0.625,1
1,6.5,0.179,13.5,6.5,0,0,2013-01-05,14,0.361,0.936,36.25,1,0.058,0.526,0.875,16.9,36.075,0.012,curryst01,0.023,0.475,0.081,0.562,0.5
2,5.667,0.152,16.333,7,0,0,2013-01-09,13.667,0.348,0.945,37.167,1,0.063,0.502,0.583,15.867,37.556,0.016,curryst01,0.032,0.483,0.087,0.518,0.333
3,7.25,0.183,17.75,7.25,0.25,0.006,2013-01-11,17.938,0.437,1.02,41.438,1.25,0.065,0.456,0.688,17.15,39.067,0.012,curryst01,0.041,0.456,0.071,0.46,0.5
4,7.2,0.18,17.8,7.6,0.2,0.005,2013-01-13,16.75,0.408,1.033,42.05,1.2,0.052,0.487,0.55,18.54,39.457,0.01,curryst01,0.038,0.521,0.062,0.457,0.4


In [22]:
data1 = pd.read_csv("data/bb_reference/2016to2019Data_merged.csv", encoding = 'latin-1')
data2 = pd.read_csv("data/bb_reference/2013to2015Data_merged.csv", encoding = 'latin-1')
data = data1.append(data2)
data = data.drop(['Unnamed: 0_y','Unnamed: 0'], axis = 1)
data = data.rename(columns={'Unnamed: 0_x':'gamescore_rank'})
data = clnr.transform(data)
data['date']=data['dt'].apply(lambda x: x.strftime("%Y-%m-%d"))
players = data['slug'].unique()
names = data['player'].unique()
slug_names = {}
name_slugs = {}
for row in data[['slug','player']].groupby('slug').agg('first').iterrows():
    slug_names[row[0]]=row[1]['player']
    name_slugs[row[1]['player']]=row[0]

In [44]:
len(resp)

391

In [24]:
data = data.sort_values(by='date',axis = 0)

In [76]:
player_hist = data[data['slug']==slug].iloc[1:,:]
player_hist.shape

(594, 54)

In [57]:
resp.index = player_df['date']
lag_data = player_df.sort_index(axis = 0,ascending=True).shift(1)
#lag_data = player_df.shift(1)
lag_data.head()

,assists_06,assists_per_min_06,attempted_field_goals_06,attempted_three_point_field_goals_06,blocks_06,blocks_per_min_06,date,dk_no_fg_06,dk_no_fg_per_min_06,dk_per_min_06,dk_score_06,doubles_06,dr_per_min_06,fg_pct_06,ft_pct_06,game_score_06,minutes_06,or_per_min_06,slug,steals_per_min_06,three_pct_06,turnovers_per_min_06,two_pct_06,w_06
date,,,,,,,,,,,,,,,,,,,,,,,,
2015-03-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-11,6.5,0.194,16.333,9.333,0.5,0.015,2015-03-09,16.583,0.491,1.315,44.5,1.167,0.064,0.496,0.806,20.067,33.683,0.014,curryst01,0.064,0.625,0.123,0.376,0.833
2015-03-14,7.5,0.226,15.167,9.167,0.5,0.015,2015-03-11,18.458,0.552,1.248,41.375,1.167,0.066,0.434,0.639,16.7,33.069,0.019,curryst01,0.065,0.545,0.116,0.331,0.833
2015-03-16,8.167,0.254,15,8.833,0.5,0.015,2015-03-14,20.833,0.646,1.351,43.583,1.333,0.073,0.44,0.639,18.133,32.681,0.019,curryst01,0.078,0.561,0.099,0.315,1
2015-03-18,7.833,0.25,14.667,7.667,0.167,0.005,2015-03-16,20.292,0.643,1.349,42.708,1.167,0.089,0.437,0.722,18.117,32.028,0.019,curryst01,0.084,0.54,0.107,0.377,1


In [55]:
player_df.sort_index(ascending=True,axis = 0)

,assists_06,assists_per_min_06,attempted_field_goals_06,attempted_three_point_field_goals_06,blocks_06,blocks_per_min_06,date,dk_no_fg_06,dk_no_fg_per_min_06,dk_per_min_06,dk_score_06,doubles_06,dr_per_min_06,fg_pct_06,ft_pct_06,game_score_06,minutes_06,or_per_min_06,slug,steals_per_min_06,three_pct_06,turnovers_per_min_06,two_pct_06,w_06
date,,,,,,,,,,,,,,,,,,,,,,,,
2015-03-09,6.5,0.194,16.333,9.333,0.5,0.015,2015-03-09,16.583,0.491,1.315,44.5,1.167,0.064,0.496,0.806,20.067,33.683,0.014,curryst01,0.064,0.625,0.123,0.376,0.833
2015-03-11,7.5,0.226,15.167,9.167,0.5,0.015,2015-03-11,18.458,0.552,1.248,41.375,1.167,0.066,0.434,0.639,16.7,33.069,0.019,curryst01,0.065,0.545,0.116,0.331,0.833
2015-03-14,8.167,0.254,15,8.833,0.5,0.015,2015-03-14,20.833,0.646,1.351,43.583,1.333,0.073,0.44,0.639,18.133,32.681,0.019,curryst01,0.078,0.561,0.099,0.315,1
2015-03-16,7.833,0.25,14.667,7.667,0.167,0.005,2015-03-16,20.292,0.643,1.349,42.708,1.167,0.089,0.437,0.722,18.117,32.028,0.019,curryst01,0.084,0.54,0.107,0.377,1
2015-03-18,8.667,0.28,14.667,7.333,0,0,2015-03-18,21.125,0.675,1.345,42.292,1.333,0.095,0.406,0.75,17.25,31.828,0.025,curryst01,0.073,0.491,0.113,0.388,1
2015-03-20,9.833,0.326,16,8.667,0,0,2015-03-20,22.417,0.734,1.44,44.25,1.5,0.102,0.39,0.75,17.65,31.006,0.025,curryst01,0.057,0.358,0.111,0.412,1
2015-03-21,9.5,0.319,15,7.5,0,0,2015-03-21,21.292,0.71,1.364,40.792,1.5,0.098,0.367,0.833,16.2,30.417,0.016,curryst01,0.054,0.352,0.092,0.375,1
2015-03-23,8.667,0.3,14.833,7.667,0,0,2015-03-23,20.458,0.703,1.467,42.792,1.5,0.102,0.418,1,18.833,29.469,0.017,curryst01,0.061,0.432,0.09,0.396,1
2015-03-24,8.5,0.279,16,7.5,0,0,2015-03-24,19.25,0.629,1.386,42.833,1.5,0.093,0.427,1,19.017,30.886,0.017,curryst01,0.047,0.425,0.093,0.432,1


In [31]:
lag_data.index

Index(['2013-01-02', '2013-01-04', '2013-01-06', '2013-01-08', '2013-01-10',
       '2013-01-12', '2013-01-14', '2013-01-16', '2013-01-17', '2013-01-23',
       ...
       '2016-12-29', '2016-12-31', '2017-01-02', '2017-01-04', '2017-01-06',
       '2017-01-08', '2017-01-10', '2017-01-11', '2017-01-13', '2017-01-16'],
      dtype='object', name='date', length=390)

In [33]:
player_hist['date']

445      2013-01-04
879      2013-01-06
1134     2013-01-08
1450     2013-01-10
1765     2013-01-12
2092     2013-01-14
2396     2013-01-16
2612     2013-01-17
3406     2013-01-23
3698     2013-01-25
4141     2013-01-27
4526     2013-01-30
4817     2013-02-01
5240     2013-02-03
5300     2013-02-04
5584     2013-02-06
5903     2013-02-08
6253     2013-02-10
6604     2013-02-12
6979     2013-02-14
7230     2013-02-20
7446     2013-02-21
7759     2013-02-23
7900     2013-02-24
8175     2013-02-26
8612     2013-03-01
8940     2013-03-03
9133     2013-03-04
9351     2013-03-06
9674     2013-03-08
            ...    
22498    2018-12-08
22499    2018-12-10
22500    2018-12-13
22501    2018-12-15
22502    2018-12-16
22503    2018-12-18
22504    2018-12-21
22505    2018-12-23
22506    2018-12-25
22507    2019-01-31
22508    2019-02-05
22509    2019-02-07
22510    2019-02-10
22511    2019-02-12
22512    2019-02-21
22513    2019-02-23
22514    2019-02-25
22515    2019-02-27
22516    2019-03-01
